In [ ]:
import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xml.etree.ElementTree as ET 
import matplotlib.pyplot as plt, zipfile 
from PIL import Image 
import dask.array as da
import h5py
import tqdm

import os
print(os.listdir("../input"))



In [ ]:
dirlist = os.listdir(".")

In [ ]:
# NN related imports
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.models import Model
from keras.optimizers import SGD, Adam
from keras import regularizers
from keras.layers import Layer, Input, Conv2D, MaxPooling2D,AveragePooling2D, UpSampling2D, Conv2DTranspose,BatchNormalization
from keras.layers import Dense, Dropout, Flatten, Reshape , LeakyReLU , Lambda, PReLU, Concatenate
from keras.initializers import RandomNormal, Constant

In [ ]:
# MODEL HYPERPARAMETERS.
LATENT_DIMS = 100
ACTIVATION = 'relu'
BATCH_SIZE=256
EPOCHS=100
LR=0.0005

# OTHER CONSTANTS.
CHUNK_SIZE = 20*1024
LOSS_FACTOR = 10000

OUT_DIR = "./cropped_images/"

In [ ]:
# extract the dog images and annotations into working directory.
if "croppedImages.h5" not in dirlist:
    import zipfile

    with zipfile.ZipFile("../input/generative-dog-images/all-dogs.zip","r") as z:
        z.extractall(".")
    with zipfile.ZipFile("../input/generative-dog-images/Annotation.zip","r") as z:
        z.extractall(".")

In [ ]:
if "croppedImages.h5" not in dirlist:
    PATH = "./all-dogs/"
    # get name of all dog image files.
    imageNames = os.listdir(PATH)

In [ ]:
# crop the images based on bounding boxes given. Then resize (and crop if necessary) to 64*64*3
if "croppedImages.h5" not in dirlist:
    PATH_ANNO = './Annotation/'
    breeds = os.listdir(PATH_ANNO)
    imagesInput = np.zeros((len(imageNames)*2,64,64,3))
    images_breed = []
    i = 0
    print(imagesInput.shape)
    for breed in breeds:
        for dog in os.listdir(PATH_ANNO+breed):
            tree = ET.parse(PATH_ANNO + breed + '/' + dog)
            root = tree.getroot()
            try: img = Image.open(PATH + root.find('filename').text +'.jpg')
            except: continue
            for obj in root.findall('object'):
                bndbox = obj.find('bndbox')
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)
                img_crop = img.crop((xmin, ymin, xmax, ymax))
                w = img_crop.size[0]; h = img_crop.size[1];
                a=0; b=0
                if w<h:
                    w2 = 64; h2 = int((64/w)*h)
                    #b = np.random.randint(0,(h2-64)) if (h2-64 > 0) else 0
                else:
                    h2 = 64; w2 = int((64/h)*w)
                    #a = np.random.randint(0,(w2-64)) if (w2-64 > 0) else 0
                img_crop = img_crop.resize((w2,h2), Image.ANTIALIAS)
                img_crop = img_crop.crop((0+a, 0+b, 64+a, 64+b))
                imagesInput[i,:,:,:] = np.asarray(img_crop)
                images_breed.append(obj.find('name').text)
                i += 1
    imagesInput = imagesInput[:i,:,:,:]   
    print(imagesInput.shape)
    flip_imagesInput = np.flip(imagesInput,2)
    imagesInput = np.vstack((imagesInput,flip_imagesInput))
    flip_imagesInput = None
    images_breed = images_breed + images_breed
#     imagesInput = imagesInput / (255 / 2) - 1
    imagesInput = imagesInput / 255
#     print(imagesInput.shape,len(images_breed))

####################################
#     # dask enhancement.
    
#     # write to file.
    
#     with h5py.File('croppedImages.h5', 'w') as h5f:
#         h5f.create_dataset('croppedImages', data=imagesInput)
#         h5f.close()

# #     del imagesInput


In [ ]:
# write each image as a file to the directory.
for i in range(imageInput.size[0]):
    try: img = Image.open(PATH + root.find('filename').text +'.jpg')
                except: continue

In [ ]:
print(imagesInput[0])
plt.imshow(imagesInput[0])

In [ ]:
ii = imagesInput[1]/255
plt.imshow(ii)
print(ii)

In [ ]:
print(ii)

In [ ]:
imagesInput = imagesInput/255

In [ ]:
# read the saved cropped images as dask array
h5f = h5py.File('croppedImages.h5','r')
# b = h5f["croppedImages"]
daskInput = da.from_array(h5f["croppedImages"], CHUNK_SIZE)

In [ ]:
daskInput.visualize()

In [ ]:
# print a random image to check.
rnd = np.random.randint(0,imagesInput.shape[0])
plt.imshow(imagesInput[rnd])
print(imagesInput[0].shape)

# print a random image to check.
# rnd = np.random.randint(0,CHUNK_SIZE)
# plt.imshow(daskInput[rnd])
# print(daskInput[0].shape)

In [ ]:
# np.array()

In [ ]:
class Sampling(Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


In [ ]:
# noise generator to get latent representation.
def gen_noise(batch_size=1):
#     noise = z_mean + tf.exp(0.5 * z_log_var) * epsilon
    noise = np.random.normal(0,1,size=(batch_size, LATENT_DIMS))
    return noise

In [ ]:
init = RandomNormal(mean=0.0, stddev=0.02)

# encoder

img_input = Input(shape=(64, 64, 3,))
x = Conv2D(32, (5,5), strides=2, padding='same', kernel_initializer=init)(img_input)
# x = PReLU(alpha_initializer=Constant(value=0.3))(x)
x = LeakyReLU()(x)

x = Conv2D(64, (2, 2), strides=2, padding='same', kernel_initializer=init)(x)
# x = PReLU(alpha_initializer=Constant(value=0.3))(x)
x = LeakyReLU()(x)

x = Flatten()(x)

x = Dense(64, activation=ACTIVATION)(x)

z_mean = Dense(LATENT_DIMS, name="z_mean")(x)
z_log_var = Dense(LATENT_DIMS, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = Model(img_input, [z_mean, z_log_var, z], name="encoder")
encoder.summary()


In [ ]:
# decoder
code_input = Input(shape=(LATENT_DIMS,))
x = Dense(16*16*64, activation=ACTIVATION)(code_input)
x = Reshape((16, 16, 64))(x)

x = Conv2DTranspose(64, (2, 2), strides=2, padding="same", kernel_initializer=init)(x)
# x = PReLU(alpha_initializer=Constant(value=0.3))(x)
x = LeakyReLU()(x)
x = Conv2DTranspose(32, (5, 5), strides=2, padding="same", kernel_initializer=init)(x)
# x = PReLU(alpha_initializer=Constant(value=0.3))(x)
x = LeakyReLU()(x)

x = Conv2DTranspose(3, (1,1), strides=1, padding="same", kernel_initializer=init, activation="sigmoid")(x)

decoder = Model(code_input, x, name="decoder")
decoder.summary()

In [ ]:
class VAE(Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        
    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = encoder(data)
            reconstruction = decoder(z)
            reconstruction_loss = tf.reduce_mean(
#                 keras.losses.binary_crossentropy(data, reconstruction)
                keras.losses.mean_squared_error(data, reconstruction)
            )
            reconstruction_loss *= 64*64*3 # check later.
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
#             kl_loss = tf.reduce_mean(kl_loss)
            kl_loss = tf.reduce_sum(kl_loss)
            kl_loss *= -0.5
        
#             def r_loss(y_true, y_pred):
#                 return tf.keras.backend.mean(tf.square(y_true - y_pred), axis=[1,2,3])
            
#             rec_loss = r_loss(data, reconstruction)
            
            total_loss = reconstruction_loss + kl_loss
#             total_loss = LOSS_FACTOR*rec_loss + kl_loss
#             total_loss = kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }
            

In [ ]:
# # training
vae = VAE(encoder, decoder)
vae.compile(optimizer=Adam())

noise = gen_noise()
loss, reconstruction_loss, kl_loss = [] , [], []
progress = []
for i in tqdm.tqdm(range(0, EPOCHS, 5)):
    history = vae.fit(imagesInput, epochs=5, batch_size=BATCH_SIZE)
    loss = history.history["loss"]
    reconstruction_loss = history.history["reconstruction_loss"]
    kl_loss = history.history["kl_loss"]
    
    progress.append(np.array(decoder.predict(noise)[0]))

# # dask enhancement.

# # training
# vae = VAE(encoder, decoder)
# vae.compile(optimizer=Adam(lr=LR))

# noise = gen_noise()
# loss, reconstruction_loss, kl_loss = [] , [], []
# progress = []
# for i in tqdm.tqdm(range(1, EPOCHS + 1)):
#     k = 0
#     loss_acc, reconstruction_loss_acc, kl_loss_acc = 0, 0, 0
#     for j in range(len(daskInput.chunks[0])):
#         history = vae.fit(daskInput[k: k + CHUNK_SIZE, :, :, :], epochs=1, batch_size=BATCH_SIZE)
#         k += CHUNK_SIZE
#         loss_acc += history.history["loss"][0]
#         reconstruction_loss_acc += history.history["reconstruction_loss"][0]
#         kl_loss_acc += history.history["kl_loss"][0]
    
#     loss_acc /= j
#     reconstruction_loss_acc /= j
#     kl_loss_acc /= j
    
#     loss.append(loss_acc)
#     reconstruction_loss.append(reconstruction_loss_acc)
#     kl_loss.append(kl_loss_acc)
    
#     progress.append(np.array(decoder.predict(noise)[0]))


In [ ]:
# save model.
vae.save("vae_dog_images")

In [ ]:
# show progress
columns = 6 ; rows = min(6,(len(progress) // columns) + 1);
fig=plt.figure(figsize=(32, 5 * rows))
j=0
for i in range(0 , min(36,len(progress))):
#     print(progress[int(j)])
    fig.add_subplot(rows,columns,i+1)
    plt.imshow(progress[int(j)])
    j += max(1,len(progress) / 36)
plt.show()

In [ ]:
print(progress)

In [ ]:
print(daskInput[0,:,:,:])
n = np.array(daskInput[44,:,:,:])
print(n)
plt.imshow(n)
plt.show()


In [ ]:
print(n.shape)
nn = np.array([n])
print(nn.shape)
code = encoder.predict(nn)
code

In [ ]:

gen_img = decoder(code)[0]
plt.imshow(gen_img)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(loss)
plt.plot(reconstruction_loss)
plt.plot(kl_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['tota loss', 'reconstruction loss', 'kl loss'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt


def plot_latent(encoder, decoder):
    # display a n*n 2D manifold of digits
    n = 30
    digit_size = 64
    scale = 2.0
    figsize = 15
    figure = np.zeros((digit_size * n, digit_size * n, 3*n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-scale, scale, n)
    grid_y = np.linspace(-scale, scale, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[
                i * digit_size : (i + 1) * digit_size,
                j * digit_size : (j + 1) * digit_size,
            ] = digit

    plt.figure(figsize=(figsize, figsize))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range + 1
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap="Greys_r")
    plt.show()


plot_latent(encoder, decoder)